In [2]:
import pandas as pd
import seaborn
dataset=seaborn.load_dataset("tips")

In [3]:
dataset.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [5]:
dataset["sex"].unique()

['Female', 'Male']
Categories (2, object): ['Male', 'Female']

In [7]:
dataset.shape

(244, 7)

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
encoder = LabelEncoder()

In [12]:
dataset["time"] = encoder.fit_transform(dataset["time"])

In [13]:
dataset["time"].value_counts()

0    176
1     68
Name: time, dtype: int64

In [16]:
#segregate dependent and independent features
x = dataset.drop("time", axis=1)
y = dataset["time"]

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [19]:
dataset.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    int64   
 6   size        244 non-null    int64   
dtypes: category(3), float64(2), int64(2)
memory usage: 8.9 KB


In [29]:
categorical_column = ["sex", "smoker", "day"]
numeric_column = ["total_bill", "tip", "size"]

In [24]:
dataset[category_column]

,sex,smoker,day
0,Female,No,Sun
1,Male,No,Sun
2,Male,No,Sun
3,Male,No,Sun
4,Female,No,Sun
...,...,...,...
239,Male,No,Sat
240,Female,Yes,Sat
241,Male,Yes,Sat
242,Male,No,Sat


In [26]:
dataset[numeric_column]

,total_bill,tip,size
0,16.99,1.01,2
1,10.34,1.66,3
2,21.01,3.50,3
3,23.68,3.31,2
4,24.59,3.61,4
...,...,...,...
239,29.03,5.92,3
240,27.18,2.00,2
241,22.67,2.00,2
242,17.82,1.75,2


In [28]:
#handling numerical features
numeric_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")), #handling missing values
        ("scaler", StandardScaler()) #scaling features
    ]
)
#handling categorical features
categoric_pipeline = Pipeline(   
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), #handling missing values
        ("onehotencoder", OneHotEncoder()) #conver categorical values into numeric
    ]
)

In [30]:
preprocessor = ColumnTransformer([
    ("num_pipeline",numeric_pipeline,numeric_column),
    ("cat_pipeline",categoric_pipeline,categorical_column)
])  

In [32]:
X_train=preprocessor.fit_transform(X_train)

In [34]:
X_test=preprocessor.transform(X_test)

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [40]:
models = {
    "Random_Forest":RandomForestClassifier(),
    "Decision_Tree":DecisionTreeClassifier(),
    "SVC":SVC()
}

In [52]:
#base model
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc_score = round(accuracy_score(y_test,y_pred)*100,4)
        report[list(models.keys())[i]] = acc_score
    return report
    

In [53]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random_Forest': 95.9184, 'Decision_Tree': 93.8776, 'SVC': 95.9184}

In [60]:
#hyperparameter tunning
classifier = RandomForestClassifier()
params = {
    "max_depth":[2,3,5,8,12],
    "n_estimators":[100,200,300,600],
    "criterion":["gini","entropy"],
    "max_features":["sqrt","log2"]
}


In [61]:
from sklearn.model_selection import GridSearchCV

In [62]:
tunning_model=GridSearchCV(classifier, params, scoring="accuracy", cv=5,verbose=3,n_jobs=-1)

In [63]:
tunning_model.fit(X_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 8, 12],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 600]},
             scoring='accuracy', verbose=3)

In [65]:
tunning_model.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [66]:
classifier = RandomForestClassifier(criterion='gini',max_depth=2,max_features='sqrt',n_estimators=100)

In [67]:
classifier.fit(X_train,y_train)

RandomForestClassifier(max_depth=2, max_features='sqrt')

In [68]:
y_pred=classifier.predict(X_test)

In [69]:
accuracy_score(y_test,y_pred)

1.0

[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=200;, score=0.974 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=300;, score=0.949 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=100;, score=0.974 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=200;, score=0.974 total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=300;, score=0.974 total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=600;, score=0.923 total time=   0.7s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=600;, score=0.974 total time=   0.7s
[CV 2/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=300;, score=0.949 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=600;, score=0.949 total time=   0.7s
[